In [85]:
import json
import re
from urllib.request import urlopen
import json
import html2text
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [4]:
'''
Collect all comments by the post
Only keep posts with more than 5 comments
Takes about 3 minutes to run

To get the reddit data (chosen arbitrarily), go to the data dir and run

mkdir reddit
cd reddit
wget https://files.pushshift.io/reddit/comments/RC_2009-05.bz2
bzip2 -d RC_2009-05.bz2
'''

comments_by_post = {}
with open('../data/reddit/RC_2009-05', 'r') as f:
    for line in f:
        d = json.loads(line)
        link_id = d['link_id']
        if link_id not in comments_by_post:
            comments_by_post[link_id] = {}
        d['body'] = html2text.html2text(d['body'])
        comments_by_post[link_id][d['id']] = d

for key in list(comments_by_post.keys()):
    if len(comments_by_post[key]) < 5:
        del comments_by_post[key]

In [35]:
def collect_ancestors(comments: dict, comment_id: str) -> list:
    '''
    For a given list of comments and a comment in the list, reconstruct a path to the top-level comment
    Returns a list of comment IDs

    This method isn't very efficient but good enough for now
    '''
    ancestors = []
    while True:
        
        if comment_id[:2] == 't3': 
            # refers to a link (top-level comment)
            # means we've reached the top of the chain
            return ancestors[::-1]

        if comment_id[:2] == 't1':
            comment_id = comment_id[3:]


        try:
            # there is an error here sometimes where the comment id is not present in the list
            # probably fine for now, but may need to address in the future
            old_comment_id = comment_id
            comment_id = comments[comment_id]['parent_id']
            ancestors.append(old_comment_id)
        except:
            return ancestors[::-1]

In [90]:
'''
Cycle through all posts and comments, find any URL mentions, and save the mention location + the comment's ancestors
'''
all_urls = []
for post_id in comments_by_post:
    for comment_id in comments_by_post[post_id]:
        # first check if post body contains URL
        urls = re.findall(r'(https?://\S+)', comments_by_post[post_id][comment_id]['body'])
        if urls != []:
            ancestors = collect_ancestors(comments_by_post[post_id], comment_id)
            for url in urls:
                # heuristics for parsing errors
                url = re.sub('\)', '', url)
                url = re.sub('\]', '', url)
                
                all_urls.append({'post_id': post_id, 'comment_id': comment_id, 'url': url, 'ancestors': ancestors})

urls_with_context = [x for x in all_urls if len(x['ancestors']) > 2]

In [84]:
'''
Loops through all ancestors of a URL comment and returns the chain up to to the top comment
'''

def display_context(url_obj: dict) -> list:
    post_id = url_obj['post_id']
    context = []
    for ancestor in url_obj['ancestors']:
        context.append(comments_by_post[post_id][ancestor]['body'])
    return context

In [67]:
'''
Simple method to scrape text from URLs. Not very robust. Need to handle exceptions. YouTube links take very long
'''

def scrape(url: str) -> str:
    try:
        html = urlopen(url).read()
    except Exception as e:
        return e

    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

In [98]:
# test some basic outputs

url_finding = urls_with_context[7]

print(display_context(url_finding))
print(scrape(url_finding['url'])[:1000])

["JKD Concepts would say he didn't do what's right. Ideally you want to\nannihilate your opponent, and then run like hell, but that gets you into jail\neven faster :(.\n\n", "What's JKD?\n\n", '[Jeet Kun Do](http://en.wikipedia.org/wiki/Jeet_Kune_Do)\n\n']
Jeet Kune Do - Wikipedia
Jeet Kune Do
From Wikipedia, the free encyclopedia
Jump to navigation
Jump to search
Hybrid martial art
This article has multiple issues. Please help improve it or discuss these issues on the talk page. (Learn how and when to remove these template messages)
This article possibly contains original research. Please improve it by verifying the claims made and adding inline citations. Statements consisting only of original research should be removed. (March 2019) (Learn how and when to remove this template message)This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources: "Jeet K

In [92]:
# explore distribution of domains (help us with parsing)
domains = {}
for url in urls_with_context:
    try:
        domain = urlparse(url['url']).netloc
    except:
        print(url['url'])
    if domain not in domains:
        domains[domain] = 0
    domains[domain] += 1

sorted_domains = sorted([(domain, domains[domain]) for domain in domains], reverse=True, key=lambda x: x[1])
print(sorted_domains[:20])

http://www.enamerique.com[(2(http://www.enamerique.net
[('en.wikipedia.org', 6109), ('www.reddit.com', 3143), ('www.youtube.com', 2591), ('imgur.com', 524), ('www.google.com', 389), ('www.amazon.com', 342), ('www.imdb.com', 251), ('scriptures.lds.org', 217), ('news.bbc.co.uk', 169), ('www.biblegateway.com', 160), ('www.nytimes.com', 148), ('www.flickr.com', 137), ('tinyurl.com', 127), ('video.google.com', 119), ('www.guardian.co.uk', 106), ('dictionary.reference.com', 104), ('xkcd.com', 103), ('images.google.com', 92), ('upload.wikimedia.org', 81), ('lokonline.com', 76)]
